# FluidAINet

In [1]:
import os
import time

import numpy as np
import tensorflow as tf

## LoadData

In [2]:
%run -i utils/fluid_loader.py

['/data/datasets/simulation_data/water/1/particles_665.csv', '/data/datasets/simulation_data/water/1/particles_1504.csv']


In [3]:
%run -i utils/preprocess.py

In [2]:
%run -i utils/model_utils.py

Tensor("ScatterNd:0", shape=(3, 40, 40, 50, 128), dtype=float32)
[<tf.Tensor 'gpu_0/MiddleAndRPN_/output/BiasAdd_1:0' shape=(?, 3) dtype=float32>]
Tensor("gpu_0/ScatterNd:0", shape=(2, 40, 40, 50, 128), dtype=float32)


In [84]:
%run -i model/rpn.py

(?, 1600, 64, 12)
(?, 1600, 64, 1)
(?, 40, 40, 64)
temp_conv shape: Tensor("MiddleAndRPN_/concat:0", shape=(?, 20, 20, 768), dtype=float32)
r_map shape: Tensor("MiddleAndRPN_/conv21/BiasAdd:0", shape=(?, 20, 20, 3), dtype=float32)
final_features shape: (?, 1200)
pred:
 Tensor("MiddleAndRPN_/output/BiasAdd:0", shape=(?, 3), dtype=float32)
********************************
Tensor("MiddleAndRPN_/Sum:0", shape=(), dtype=float32)


In [4]:
data_dir = '/data/datasets/simulation_data'
train_dir = os.path.join(data_dir, 'water')
TRAIN_FILES = get_all_frames(train_dir)
file_path = list(TRAIN_FILES)[5]

In [6]:
len(list(TRAIN_FILES))

0

In [4]:
batch_size = 25

In [6]:
singel_batch = None
for batch in iterate_data(train_dir, batch_size=batch_size):
    singel_batch = batch 
    break
    

csv


In [7]:
singel_batch[0].shape

(1, 25, 3)

In [8]:
"""
ret = (
    np.array(vox_labels),
    np.array(vox_feature),
    np.array(vox_number),
    np.array(vox_coordinate),
    np.array(vox_centroid),
    np.array(vox_k_dynamic)
)
"""
singel_batch[1].shape

(1, 58925, 64, 8)

保存目前的模型，并加载在新的graph中

In [9]:
# # freeze model
model_dir = "/data/deeplearning/FluidAiNet/save_model/default"
try:
    freeze_graph(model_dir, output_node_names="gpu_0/screen_size,concat_129,gpu_0/MiddleAndRPN_/output/BiasAdd")
except Exception as e:
    print("output_node_names not find in the graph:\n", e)

output_node_names not find in the graph:
 'NoneType' object has no attribute 'model_checkpoint_path'


In [9]:
# graph model file path
model_dir = "/data/deeplearning/FluidAiNet/save_model/default"
frozen_model_filename = os.path.join(model_dir, "../frozen_model/frozen_model.pb")

In [5]:
# load graph
# graph = load_graph(frozen_model_filename)
graph, acceleration = load_graph_with_input_map(frozen_model_filename, batch_size=batch_size)

Tensor("ScatterNd:0", shape=(25, 40, 40, 50, 128), dtype=float32)
[<tf.Tensor 'gpu_0/MiddleAndRPN_/output/BiasAdd_1:0' shape=(?, 3) dtype=float32>]


## Full Flow

In [15]:
"""
Full Flow
1. create new scatter_nd with new batch size
2. get graph and acceleration by load_graph_with_input_map
3. get tensor in graph by tensor name(pay a attention tensor should be in same graph)
4. load particles in a frame
"""
graph, acceleration = load_graph_with_input_map(frozen_model_filename, batch_size=batch_size)
"""
node name
"""
'gpu_0/feature_0'
'gpu_0/k_dynamics'
'gpu_0/centroid'
'gpu_0/coordinate'
screen_size = graph.get_tensor_by_name('gpu_0/screen_size:0')
feature = graph.get_tensor_by_name('gpu_0/feature:0')
part_feature = graph.get_tensor_by_name('gpu_0/part_feature:0')
concat_feature = graph.get_tensor_by_name('concat_129:0')
# accelaration = graph.get_tensor_by_name("gpu_0/MiddleAndRPN_/output/BiasAdd:0")
centroid = graph.get_tensor_by_name("gpu_0/centroid:0")
k_dynamics = graph.get_tensor_by_name("gpu_0/k_dynamics:0")
coordinate = graph.get_tensor_by_name("gpu_0/coordinate:0")
phase = graph.get_tensor_by_name("phase:0")
scatter_nd = graph.get_tensor_by_name('gpu_0/ScatterNd:0')
voxelwise = graph.get_tensor_by_name("gpu_0/Max_1:0")
# 需要feed新的变量
phase_1 = graph.get_tensor_by_name("phase_1:0")

"""
generate fake data with batch_size = 3
"""
def sample_Z_3D(m, n, k):
    return np.random.uniform(-8., 8., size=[m, n, k])
def sample_Z_2D(m, n):
    return np.random.uniform(-8., 8., size=[m, n])

feature_ = sample_Z_3D(7, 64, 8)
k_dynamics_ = np.array([4, 1, 2])
coordinate_ = np.array([[0, 1, 2, 3],
           [0, 1, 2, 3],
           [0, 1, 2, 3],
           [0, 1, 2, 3],
           [1, 1, 2, 3],
           [1, 1, 2, 3],
           [2, 1, 2, 3],  ])
centroid_ = np.array([[0.4, 0.4, 0.4], [0.3, 0.3, 0.3], [0.3, 0.3, 0.3]])
   
    


Tensor("ScatterNd:0", shape=(25, 40, 40, 50, 128), dtype=float32)
[<tf.Tensor 'gpu_0/MiddleAndRPN_/output/BiasAdd_1:0' shape=(?, 3) dtype=float32>]


In [16]:
def fluidnet_predict(graph, batch_size=1, singel_batch=None):
    
    
    # predict
    with tf.Session(graph=graph) as sess:
        input_dict = dict()
        input_dict[phase] = False
        input_dict[phase_1] = False
        screen_size_eval = screen_size.eval(session=sess, feed_dict={screen_size: batch_size})
        concat_feature_all = []
        count = 0
        # 如果要并行的话肯定还是要处理的，
        print("screen_size_eval:\n", screen_size_eval)
        for screen in range(screen_size_eval):
            k = k_dynamics[screen]
            concat_feature_all.append(tf.concat([part_feature[count: count+k],part_feature[count: count+k,:,:3] -centroid[screen]],axis=2))

            count += k
        concat_feature_all = tf.concat(concat_feature_all, axis=0)
        """
        what if I use fake data
        """
        option = 2
        if option == 2 and singel_batch is None:
            print("single_batch can't be None")
            return 
        if option == 1:
            input_dict[part_feature] = feature_
            input_dict[k_dynamics] = k_dynamics_
            input_dict[centroid] = centroid_
            concat_feature_eval = sess.run(concat_feature_all, input_dict)
            input_dict[feature] = concat_feature_eval
            input_dict[coordinate] = coordinate_
        #     input_dict[coordinate] = singel_batch[0]
            accelaration_eval = sess.run(acceleration[0], feed_dict=input_dict)
        else:
            input_dict[part_feature] = singel_batch[1][0]
            input_dict[k_dynamics] = singel_batch[5][0]
            input_dict[centroid] = singel_batch[4][0]
            concat_feature_eval = sess.run(concat_feature_all, input_dict)
            input_dict[feature] = concat_feature_eval
            input_dict[coordinate] = singel_batch[3][0]
            input_dict[phase] = False
        #     input_dict[coordinate] = singel_batch[0]
            accelaration_eval = sess.run(acceleration[0], feed_dict=input_dict)
            
        return accelaration_eval
 

In [ ]:
"""
frame to batch
"""
start = time.clock()
result = []
for batch in iterate_single_frame(train_dir, file_path, batch_size=batch_size):
    accelaration_eval = fluidnet_predict(graph, batch_size=batch_size, singel_batch=batch)
    result.append(accelaration_eval)
print('duration:\n', time.clock()-start)

csv
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen

screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_siz

screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_siz

Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Traceback (most recent call last):
Process ForkPoolWorker-14:
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/f1001/.pyenv/versions/3.5.4/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "

screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25
screen_size_eval:
 25


In [17]:
accelaration_eval = fluidnet_predict(graph, batch_size=batch_size, singel_batch=singel_batch)

screen_size_eval:
 25


In [18]:
accelaration_eval.shape

(25, 3)

In [20]:
result = []

In [22]:
result.extend(accelaration_eval)

[array([ -1.49290085,   0.13700485, -23.88957787], dtype=float32),
 array([ -1.49290085,   0.13700485, -23.88957787], dtype=float32),
 array([ -1.49021721,   0.14236933, -23.99931717], dtype=float32),
 array([ -1.49021721,   0.14236933, -23.99931717], dtype=float32),
 array([ -1.48733687,   0.14902812, -24.09713364], dtype=float32),
 array([ -1.48733687,   0.14902812, -24.09713364], dtype=float32),
 array([ -1.48436522,   0.15466624, -24.17471886], dtype=float32),
 array([ -1.48436522,   0.15466624, -24.17471886], dtype=float32),
 array([ -1.48328257,   0.15874499, -24.2313118 ], dtype=float32),
 array([ -1.48328257,   0.15874499, -24.2313118 ], dtype=float32),
 array([ -1.48043323,   0.16298372, -24.25707245], dtype=float32),
 array([ -1.48043323,   0.16298372, -24.25707245], dtype=float32),
 array([ -1.48060989,   0.16921085, -24.27262688], dtype=float32),
 array([ -1.48060989,   0.16921085, -24.27262688], dtype=float32),
 array([ -1.49104095,   0.17247421, -24.27692223], dtype=float

## Quick Test

In [88]:
prefix = ''
with tf.gfile.GFile(frozen_model_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

# Then, we import the graph_def into a new Graph and returns it
graph_out = graph
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name=prefix)
    coordinate = graph.get_tensor_by_name("gpu_0/coordinate:0")
    voxelwise = graph.get_tensor_by_name("gpu_0/Max_1:0")
    scatter_nd = graph.get_tensor_by_name("gpu_0/ScatterNd:0")
    # holder = tf.placeholder(tf.float32,shape=(3, 40, 40, 50, 1))
    new_scatter_nd = tf.scatter_nd(coordinate, voxelwise, shape=[batch_size, 40, 40, 50, 128])
    print(new_scatter_nd)
    conv3D = graph.get_tensor_by_name('gpu_0/MiddleAndRPN_/conv1/Conv3D:0')
    # print(conv3D)
    # conv3D.input = new_scatter_nd
    # print(conv3D.input.shape)
    # graph.graph_place()
    # The name var will prefix every op/nodes in your graph
    # Since we load everything in a new graph, this is not needed
#with tf.Graph().as_default() as graph:
    #scatter_nd = graph.get_tensor_by_name('gpu_0/ScatterNd:0')
    # saver = tf.train.import_meta_graph(graph_def, name=prefix, input_map={'gpu_0/ScatterNd': holder})
    #z = tf.import_graph_def(graph_def, name=prefix, input_map={'gpu_0/ScatterNd:0': new_scatter_nd},return_elements=["gpu_0/MiddleAndRPN_/conv1/Conv3D:0"])
    accerlation = tf.import_graph_def(graph_def, name=prefix, input_map={'gpu_0/ScatterNd:0': new_scatter_nd}, return_elements=["gpu_0/MiddleAndRPN_/conv3/Conv3D:0"])
    print(accerlation)
    graph_out = graph
    print(graph)

Tensor("ScatterNd:0", shape=(3, 40, 40, 50, 128), dtype=float32)
[<tf.Tensor 'gpu_0/MiddleAndRPN_/conv3/Conv3D_1:0' shape=(3, 40, 40, 12, 64) dtype=float32>]


In [29]:
import math
math.floor(9/2)

4

In [89]:
# graph_out.get_operations()

In [67]:
voxelwise

<tf.Tensor 'gpu_0/Max_1:0' shape=(?, 128) dtype=float32>

In [29]:
conv3d_1 = graph.get_tensor_by_name('gpu_0/MiddleAndRPN_/conv1/Conv3D:0')

In [30]:
conv3d_1

<tf.Tensor 'gpu_0/MiddleAndRPN_/conv1/Conv3D:0' shape=(4, 40, 40, 25, 64) dtype=float32>

In [7]:
# cheack nodes in gaph
with tf.Session(graph=graph) as sess:
#     sess.run(tf.global_variables_initializer())
    print(all_nodes(""))

['phase', 'gpu_0/part_feature', 'gpu_0/centroid', 'gpu_0/k_dynamics', 'gpu_0/screen_size', 'gpu_0/feature', 'gpu_0/coordinate', 'gpu_0/Max/reduction_indices', 'gpu_0/Max', 'gpu_0/NotEqual/y', 'gpu_0/NotEqual', 'VFE-1/kernel', 'VFE-1/kernel/read', 'VFE-1/bias', 'VFE-1/bias/read', 'gpu_0/VFE-1/Tensordot/Shape', 'gpu_0/VFE-1/Tensordot/Rank', 'gpu_0/VFE-1/Tensordot/axes', 'gpu_0/VFE-1/Tensordot/GreaterEqual/y', 'gpu_0/VFE-1/Tensordot/GreaterEqual', 'gpu_0/VFE-1/Tensordot/Cast', 'gpu_0/VFE-1/Tensordot/mul', 'gpu_0/VFE-1/Tensordot/Less/y', 'gpu_0/VFE-1/Tensordot/Less', 'gpu_0/VFE-1/Tensordot/Cast_1', 'gpu_0/VFE-1/Tensordot/add', 'gpu_0/VFE-1/Tensordot/mul_1', 'gpu_0/VFE-1/Tensordot/add_1', 'gpu_0/VFE-1/Tensordot/range/start', 'gpu_0/VFE-1/Tensordot/range/delta', 'gpu_0/VFE-1/Tensordot/range', 'gpu_0/VFE-1/Tensordot/ListDiff', 'gpu_0/VFE-1/Tensordot/Gather', 'gpu_0/VFE-1/Tensordot/Gather_1', 'gpu_0/VFE-1/Tensordot/Const', 'gpu_0/VFE-1/Tensordot/Prod', 'gpu_0/VFE-1/Tensordot/Const_1', 'gpu_0/V

In [70]:
import time 
def how_time(f):
    def wrapper():
        start = time.clock()
        f()
        duration = time.clock() - start
        print(duration)
    return wrapper

需要feed新的变量

In [93]:
# @how_time
def hello():
    print('hello world')

In [94]:
def hello():
    print('hello world')
hello()

TypeError: 'NoneType' object is not callable

In [10]:
def sample_Z_3D(m, n, k):
    return np.random.uniform(-8., 8., size=[m, n, k])
def sample_Z_2D(m, n):
    return np.random.uniform(-8., 8., size=[m, n])

In [11]:
feature_ = sample_Z_3D(7, 64, 8)
k_dynamics_ = np.array([4,1, 2])
coordinate_ = np.array([[0, 1, 2, 3],
               [0, 1, 2, 3],
               [0, 1, 2, 3],
               [0, 1, 2, 3],
               [1, 1, 2, 3],
               [1, 1, 2, 3],
               [2, 1, 2, 3],  ])
centroid_ = np.array([[0.4, 0.4, 0.4], [0.3, 0.3, 0.3], [0.3, 0.3, 0.3]])

In [12]:
scatter_nd.shape
# scatter_nd.set_shape([batch_size, 40, 40, 50, 128])

TensorShape([Dimension(2), Dimension(40), Dimension(40), Dimension(50), Dimension(128)])

In [61]:
"""
"""
with tf.Session(graph=graph) as sess:
    input_dict = dict()
    input_dict[phase] = False
    input_dict[phase_1] = False
    screen_size_eval = screen_size.eval(session=sess, feed_dict={screen_size: batch_size})
    concat_feature_all = []
    count = 0
    # 如果要并行的话肯定还是要处理的，
    print("screen_size_eval:\n", screen_size_eval)
    for screen in range(screen_size_eval):
        k = k_dynamics[screen]
        concat_feature_all.append(tf.concat([part_feature[count: count+k],part_feature[count: count+k,:,:3] -centroid[screen]],axis=2))
        
        count += k
    concat_feature_all = tf.concat(concat_feature_all, axis=0)
    """
    what if I use fake data
    """
    option = 2
    if option == 1:
        input_dict[part_feature] = feature_
        input_dict[k_dynamics] = k_dynamics_
        input_dict[centroid] = centroid_
        concat_feature_eval = sess.run(concat_feature_all, input_dict)
        input_dict[feature] = concat_feature_eval
        input_dict[coordinate] = coordinate_
    #     input_dict[coordinate] = singel_batch[0]
        accelaration_eval = sess.run(acceleration[0], feed_dict=input_dict)
    else:
        input_dict[part_feature] = singel_batch[1][0]
        input_dict[k_dynamics] = singel_batch[5][0]
        input_dict[centroid] = singel_batch[4][0]
        concat_feature_eval = sess.run(concat_feature_all, input_dict)
        input_dict[feature] = concat_feature_eval
        input_dict[coordinate] = singel_batch[3][0]
        input_dict[phase] = False
    #     input_dict[coordinate] = singel_batch[0]
        accelaration_eval = sess.run(acceleration[0], feed_dict=input_dict)
    
    
    
    
    
    

screen_size_eval:
 25


In [62]:
concat_feature_eval.shape

(58925, 64, 11)

In [63]:
np.unique(singel_batch[3][0][:,0])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [64]:
accelaration_eval.shape

(20, 3)

In [65]:
accelaration_eval

array([[ -1.48863697,   0.13510001, -23.80851936],
       [ -1.48863697,   0.13510001, -23.80851936],
       [ -1.48627353,   0.14016443, -23.91655159],
       [ -1.48627353,   0.14016443, -23.91655159],
       [ -1.48332763,   0.14660889, -24.01422501],
       [ -1.48332763,   0.14660889, -24.01422501],
       [ -1.48014379,   0.15262049, -24.09187698],
       [ -1.48014379,   0.15262049, -24.09187698],
       [ -1.47810173,   0.15674216, -24.14888573],
       [ -1.47810173,   0.15674216, -24.14888573],
       [ -1.47478485,   0.16055459, -24.17424202],
       [ -1.47478485,   0.16055459, -24.17424202],
       [ -1.47417378,   0.1671055 , -24.18912315],
       [ -1.47417378,   0.1671055 , -24.18912315],
       [ -1.48305917,   0.17102879, -24.1921978 ],
       [ -1.48305917,   0.17102879, -24.1921978 ],
       [ -1.49908352,   0.17454356, -24.18172264],
       [ -1.49908352,   0.17454356, -24.18172264],
       [ -1.52233815,   0.17662519, -24.15372276],
       [ -1.52233815,   0.17662

In [96]:
"""
get all files including screens and frames
"""
BASE_DIR = '/data/datasets/simulation_data'
DATA_DIR = os.path.join(BASE_DIR, 'water')

['1', '0']

In [118]:
def get_all_frames(data_dir=DATA_DIR):
    dirs = os.listdir(data_dir)
    frames = []
    for item in dirs:
        screen_path = os.path.join(data_dir, item)
        allfiles = os.listdir(screen_path)
        frames.extend(map(lambda x:os.path.join(item, x), allfiles))
    return map(lambda x:os.path.join(data_dir, x), frames)

In [123]:
TRAIN_FILES = get_all_frames(DATA_DIR)

In [126]:
for f in TRAIN_FILES:
    particles = load_data_file(f)
    break


csv


In [133]:
cols = particles.columns
index = particles[particles[cols[7]]==0].index

In [113]:
len(frames)

4528

In [141]:
# dir(index)

In [140]:
# particles.iloc[0]

In [154]:
%%writefile FuildAINetTest.py

import utils.fluid_loader as fl
BASE_DIR = '/data/datasets/simulation_data'
DATA_DIR = os.path.join(BASE_DIR, 'water')

class TestFluid(object):
    def __init__(self):
        self.data_dir = DATA_DIR
    def test_iter_data(self):
        for item in fl.iterate_data(self.data_dir):
            print(item)
        
if __name__ == '__main__':
    testf1 = TestFluid()
    testf1.test_iter_data()
        

Overwriting FuildAINetTest.py


In [169]:
import utils.fluid_loader as fl
import imp
imp.reload(fl)

<module 'utils.fluid_loader' from '/data/deeplearning/FluidAiNet/utils/fluid_loader.py'>

Fluid
(62032, 3)
2357


In [163]:
fl.iterate_data(DATA_DIR)

<generator object iterate_data at 0x7f6a9a025888>

In [182]:
%run -i utils/preprocess.py

In [195]:
BATCH_SIZE = 2
TRAIN_FILES = fl.create_train_files(BATCH_SIZE)
print(TRAIN_FILES)
pointcloud, _, _ = fl.load_data_label(TRAIN_FILES[0])
voxel_index = fluid_process_pointcloud(pointcloud, 1)

['/data/datasets/simulation_data/water/1/particles_665.csv', '/data/datasets/simulation_data/water/1/particles_1504.csv']
csv
Fluid
[[-1.      -1.       1.05335 ..., -8.21249  0.       0.     ]
 [-0.9     -1.       1.05335 ..., -8.21249  0.       0.     ]
 [-0.8     -1.       1.05335 ..., -8.21249  0.       0.     ]
 ..., 
 [ 3.8      4.       3.95    ...,  0.       1.       0.     ]
 [ 3.9      4.       3.95    ...,  0.       1.       0.     ]
 [ 4.       4.       3.95    ...,  0.       1.       0.     ]]
(62032, 3)
2357


AttributeError: 'EasyDict' object has no attribute 'VOXEL_POINT_FEATURE'

In [191]:
import config 

In [193]:
imp.reload(config)

<module 'config' from '/data/deeplearning/FluidAiNet/config.py'>

In [197]:
a = np.array([[1, 2, 3], [4, 5, 6]])

In [198]:
a -[1,1,1]

array([[0, 1, 2],
       [3, 4, 5]])

In [3]:
temp_p = np.array([1, 1])
temp_p = np.lib.pad(temp_p, (1, 1), 'constant', constant_values=(0, 0))

In [4]:
temp_p

array([0, 1, 1, 0])

In [5]:
paddings = (np.array(temp_p)).repeat(2).reshape(4, 2)

In [6]:
paddings

array([[0, 0],
       [1, 1],
       [1, 1],
       [0, 0]])

In [9]:
input = np.random.uniform(-8., 8., size=[3, 40,40, 64])

In [13]:
pad = tf.pad(input, paddings, "CONSTANT")

In [14]:
pad

<tf.Tensor 'Pad:0' shape=(3, 42, 42, 64) dtype=float64>

In [3]:
a = []
b = [1, 2]
a.extend(b)
a.extend(b)

In [4]:
a

[1, 2, 1, 2]